In [37]:
import pandas as pd
from surprise import Dataset, Reader,KNNBasic
from surprise.model_selection import train_test_split


In [ ]:
pip install scikit-surprise

In [104]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('Loan_data_k.csv')

# Display the first few rows of the DataFrame
print(df.head())


  Customer_id  Age  Gender  Income  Credit_score Credit_score_category  \
0       C0124   66    Male  108296           627                  Fair   
1       C0119   25    Male   88437           589                  Fair   
2        C044   21    Male   93447           832             Excellent   
3       C0148   55    Male  138441           845             Excellent   
4       C0196   22  Female  184615           606                  Fair   

   Loan_amount  Interest_rate  Loan_term  Loan_type  ...  Transaction_freq  \
0       225098           1.82         12  Education  ...                 4   
1       109731          10.80         60       Auto  ...                10   
2       261695          12.24         48       Auto  ...                 7   
3       368030           5.67         48       Auto  ...                 8   
4       257577           5.96         12   Business  ...                 6   

   Product_id             Product_names  Interaction_type Interaction_date  \
0       

In [105]:
df.loc[df['Product_id'] == 102, 'Product_name'] = 'Premium Checking Account'

In [106]:
interaction_type_mapping={'Viewed': 1, 'Clicked': 2, 'Purchased': 3}
df["Interaction_Type"]=df["Interaction_type"].map(interaction_type_mapping)

reader=Reader(rating_scale=(1,3))
data = Dataset.load_from_df(df[['Customer_id', 'Product_id', 'Interaction_Type']], reader)
trainset, testset = train_test_split(data, test_size=0.25)


model = KNNBasic(sim_options={'name': 'cosine', 'user_based': True})
model.fit(trainset)


Computing the cosine similarity matrix...
Done computing similarity matrix.


In [107]:
product_mapping = {
    "P001": "Savings Account",
    "P002": "Premium Checking Account",
    "P003": "Credit Card",
    "P004": "Personal Loan",
    "P005": "Home Loan",
    "P006": "Auto Loan",
    "P007": "Business Loan",
    "P008": "Student Loan",
    "P009": "Investment Fund",
    "P010": "Retirement Plan",
    "P011": "Insurance Policy",
    "P012": "Mutual Fund",
    "P013": "Bond",
    "P014": "Certificate of Deposit",
    "P015": "Home Equity Line",
    "P016": "Mortgage Refinance",
    "P017": "Business Credit Line",
    "P018": "Auto Refinance",
    "P019": "Home Improvement Loan",
    "P020": "Gold Loan",
    "P021": "Cash Credit",
    "P022": "Short-Term Loan",
    "P023": "Long-Term Loan",
    "P024": "Travel Loan",
    "P025": "Medical Loan",
    "P026": "Emergency Loan",
    "P027": "Holiday Loan",
    "P028": "Debt Consolidation Loan",
    "P029": "Small Business Loan",
    "P030": "Agricultural Loan",
    "P031": "Technology Loan",
    "P032": "Education Savings Plan",
    "P033": "Wealth Management",
    "P034": "Stock Investment",
    "P035": "Real Estate Investment",
    "P036": "International Investment",
    "P037": "Fixed Deposit",
    "P038": "Recurring Deposit",
    "P039": "Loan Against Property",
    "P040": "Gold Investment",
    "P041": "Retirement Savings",
    "P042": "High-Yield Savings Account",
    "P043": "Money Market Account",
    "P044": "Insurance Savings",
    "P045": "Pension Plan",
    "P046": "Child Education Fund",
    "P047": "Healthcare Savings",
    "P048": "Property Investment",
    "P049": "Auto Insurance"
}




In [108]:
def recommend_products(customer_id, model, interaction_data, product_mapping, n=5):
    all_products = set(interaction_data['Product_id'].unique())
    interacted_products = set(interaction_data[interaction_data['Customer_id'] == customer_id]['Product_id'])
    products_to_predict = list(all_products - interacted_products)
    predictions = [model.predict(customer_id, product_id) for product_id in products_to_predict]
    top_n_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)[:n]
    recommended_product_ids = [pred.iid for pred in top_n_predictions]
    recommended_products = pd.DataFrame({
        'Product_Id': recommended_product_ids,
        'Product_Name': [product_mapping.get(pid, 'Unknown') for pid in recommended_product_ids]
    })
    return recommended_products

Customer_Id = 'C053'
recommended_products = recommend_products(Customer_Id, model, df, product_mapping, n=5)
print(recommended_products)

predictions = model.test(testset)

  Product_Id            Product_Name
0       P020               Gold Loan
1       P010         Retirement Plan
2       P014  Certificate of Deposit
3       P003             Credit Card
4       P007           Business Loan


In [90]:
import pandas as pd
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split

# Sample Data
data_dict = {
    'user': ['U001', 'U002', 'U001', 'U003', 'U004', 'U005', 'U001', 'U003', 'U002', 'U004'],
    'item': ['I001', 'I002', 'I003', 'I001', 'I005', 'I002', 'I004', 'I003', 'I005', 'I004'],
    'rating': [5, 4, 3, 2, 1, 4, 2, 5, 3, 4]
}

# Create a DataFrame
df1 = pd.DataFrame(data_dict)

# Save the DataFrame to a CSV file
csv_file_path = 'sample_data.csv'
df1.to_csv(csv_file_path, index=False)

# Load the data
reader = Reader(line_format='user item rating', sep=',', rating_scale=(1, 5))
data1 = Dataset.load_from_df(df1[['user', 'item', 'rating']], reader=reader)

# Split the data
trainset, testset = train_test_split(data1, test_size=0.25)

# Train the KNNBasic model
model = KNNBasic(sim_options={'name': 'cosine', 'user_based': True})
model.fit(trainset)

# Generate predictions
predictions = model.test(testset)

# Precision and Recall@k
def precision_recall_at_k(predictions, k=5, threshold=3.5):
    pred_df = pd.DataFrame(predictions, columns=['uid', 'iid', 'true_r', 'est', 'details'])
    pred_df['is_relevant'] = pred_df['true_r'] >= threshold
    pred_df = pred_df.sort_values(by=['uid', 'est'], ascending=[True, False])
    top_k = pred_df.groupby('uid').head(k)

    precision = top_k.groupby('uid')['is_relevant'].mean().mean()
    recall = top_k.groupby('uid')['is_relevant'].sum() / pred_df.groupby('uid')['is_relevant'].sum()
    recall = recall.mean()
    return precision, recall

# Mean Average Precision (MAP@k)
def mean_average_precision(predictions, k=5, threshold=3.5):
    pred_df = pd.DataFrame(predictions, columns=['uid', 'iid', 'true_r', 'est', 'details'])
    pred_df['is_relevant'] = pred_df['true_r'] >= threshold
    pred_df = pred_df.sort_values(by=['uid', 'est'], ascending=[True, False])

    map_scores = []
    for uid, user_data in pred_df.groupby('uid'):
        user_data = user_data.head(k)
        if user_data['is_relevant'].sum() == 0:
            continue

        precisions = [
            user_data.iloc[:i+1]['is_relevant'].mean() for i in range(len(user_data))
        ]
        ap = sum(precisions * user_data['is_relevant']) / user_data['is_relevant'].sum()
        map_scores.append(ap)

    return sum(map_scores) / len(map_scores) if map_scores else 0

# Normalized Discounted Cumulative Gain (NDCG@k)
import numpy as np

def ndcg_at_k(predictions, k=5, threshold=3.5):
    pred_df = pd.DataFrame(predictions, columns=['uid', 'iid', 'true_r', 'est', 'details'])
    pred_df['is_relevant'] = pred_df['true_r'] >= threshold
    pred_df = pred_df.sort_values(by=['uid', 'est'], ascending=[True, False])

    ndcg_scores = []
    for uid, user_data in pred_df.groupby('uid'):
        user_data = user_data.head(k)
        if user_data['is_relevant'].sum() == 0:
            continue

        user_data['dcg'] = user_data['is_relevant'] / np.log2(user_data.index + 2)
        dcg = user_data['dcg'].sum()

        idcg = sum(1 / np.log2(i + 2) for i in range(int(user_data['is_relevant'].sum())))
        ndcg_scores.append(dcg / idcg if idcg > 0 else 0)

    return sum(ndcg_scores) / len(ndcg_scores) if ndcg_scores else 0

# Calculate Metrics
precision, recall = precision_recall_at_k(predictions, k=5)
map_score = mean_average_precision(predictions, k=5)
ndcg_score = ndcg_at_k(predictions, k=5)

# Display Metrics
print(f"Precision@5: {precision}")
print(f"Recall@5: {recall}")
print(f"MAP@5: {map_score}")
print(f"NDCG@5: {ndcg_score}")


Computing the cosine similarity matrix...
Done computing similarity matrix.
Precision@5: 1.0
Recall@5: 1.0
MAP@5: 1.0
NDCG@5: 0.7103099178571526


<ipython-input-90-c0e0cd6cc30b>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_data['dcg'] = user_data['is_relevant'] / np.log2(user_data.index + 2)
<ipython-input-90-c0e0cd6cc30b>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_data['dcg'] = user_data['is_relevant'] / np.log2(user_data.index + 2)
<ipython-input-90-c0e0cd6cc30b>:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [95]:
interaction_type_mapping={'Viewed': 1, 'Clicked': 2, 'Purchased': 3}
df["Interaction_Type"]=df["Interaction_type"].map(interaction_type_mapping)

reader=Reader(rating_scale=(1,3))
data = Dataset.load_from_df(df[['Customer_id', 'Product_id', 'Interaction_Type']], reader)
trainset, testset = train_test_split(data, test_size=0.25)
param_grid={
    "k":[5,10,15],
    "sim_options":{"name":["cosine","msd","pearson"],
    "user_based":[True,False]}
}

gs=GridSearchCV(KNNBasic,param_grid,measures=["rmse"],cv=3)
gs.fit(data)

best_knn_model = gs.best_estimator["rmse"]
best_knn_model.fit(trainset)

def recommend_products(customer_id, model, interaction_data, product_mapping, n=5):
    all_products = set(interaction_data['Product_id'].unique())
    interacted_products = set(interaction_data[interaction_data['Customer_id'] == customer_id]['Product_id'])
    products_to_predict = list(all_products - interacted_products)
    predictions = [model.predict(customer_id, product_id) for product_id in products_to_predict]
    top_n_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)[:n]
    recommended_product_ids = [pred.iid for pred in top_n_predictions]
    recommended_products = pd.DataFrame({
        'Product_Id': recommended_product_ids,
        'Product_Name': [product_mapping.get(pid, 'Unknown') for pid in recommended_product_ids]
    })
    return recommended_products

Customer_Id = 'C0124'
recommended_products = recommend_products(Customer_Id, model, df, product_mapping, n=5)
print(recommended_products)

predictions = model.test(testset)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similar

In [96]:
import pandas as pd
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split, GridSearchCV

# Sample Data
data_dict = {
    'user': ['U001', 'U002', 'U001', 'U003', 'U004', 'U005', 'U001', 'U003', 'U002', 'U004'],
    'item': ['I001', 'I002', 'I003', 'I001', 'I005', 'I002', 'I004', 'I003', 'I005', 'I004'],
    'rating': [5, 4, 3, 2, 1, 4, 2, 5, 3, 4]
}

# Create a DataFrame
df = pd.DataFrame(data_dict)

# Load the data into a Surprise dataset
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user', 'item', 'rating']], reader)

# Split the data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.25)

# Define the parameter grid for GridSearchCV
param_grid = {
    "k": [5, 10, 15, 20],  # Number of neighbors
    "sim_options": {
        "name": ["cosine", "msd", "pearson"],  # Similarity measures
        "user_based": [True, False]  # User-based or item-based collaborative filtering
    }
}

# Perform GridSearchCV
gs = GridSearchCV(KNNBasic, param_grid, measures=["rmse", "mae"], cv=3, n_jobs=-1)
gs.fit(data)  # Pass the full Surprise dataset here

# Retrieve the best model
best_knn_model = gs.best_estimator["rmse"]
print(f"Best Parameters: {gs.best_params['rmse']}")

# Train the best model on the training set
best_knn_model.fit(trainset)

# Generate predictions using the test set
predictions = best_knn_model.test(testset)



Best Parameters: {'k': 5, 'sim_options': {'name': 'cosine', 'user_based': True}}
Computing the cosine similarity matrix...
Done computing similarity matrix.
User: U003, Item: I003, True Rating: 5.0, Predicted Rating: 2.857142857142857
User: U001, Item: I001, True Rating: 5.0, Predicted Rating: 2.857142857142857
User: U001, Item: I003, True Rating: 3.0, Predicted Rating: 2.857142857142857


In [98]:
# Precision and Recall@k
def precision_recall_at_k(predictions, k=5, threshold=3.5):
    pred_df = pd.DataFrame(predictions, columns=['uid', 'iid', 'true_r', 'est', 'details'])
    pred_df['is_relevant'] = pred_df['true_r'] >= threshold
    pred_df = pred_df.sort_values(by=['uid', 'est'], ascending=[True, False])
    top_k = pred_df.groupby('uid').head(k)

    precision = top_k.groupby('uid')['is_relevant'].mean().mean()
    recall = top_k.groupby('uid')['is_relevant'].sum() / pred_df.groupby('uid')['is_relevant'].sum()
    recall = recall.mean()
    return precision, recall

# Mean Average Precision (MAP@k)
def mean_average_precision(predictions, k=5, threshold=3.5):
    pred_df = pd.DataFrame(predictions, columns=['uid', 'iid', 'true_r', 'est', 'details'])
    pred_df['is_relevant'] = pred_df['true_r'] >= threshold
    pred_df = pred_df.sort_values(by=['uid', 'est'], ascending=[True, False])

    map_scores = []
    for uid, user_data in pred_df.groupby('uid'):
        user_data = user_data.head(k)
        if user_data['is_relevant'].sum() == 0:
            continue

        precisions = [
            user_data.iloc[:i+1]['is_relevant'].mean() for i in range(len(user_data))
        ]
        ap = sum(precisions * user_data['is_relevant']) / user_data['is_relevant'].sum()
        map_scores.append(ap)

    return sum(map_scores) / len(map_scores) if map_scores else 0

# Normalized Discounted Cumulative Gain (NDCG@k)
import numpy as np

def ndcg_at_k(predictions, k=5, threshold=3.5):
    pred_df = pd.DataFrame(predictions, columns=['uid', 'iid', 'true_r', 'est', 'details'])
    pred_df['is_relevant'] = pred_df['true_r'] >= threshold
    pred_df = pred_df.sort_values(by=['uid', 'est'], ascending=[True, False])

    ndcg_scores = []
    for uid, user_data in pred_df.groupby('uid'):
        user_data = user_data.head(k)
        if user_data['is_relevant'].sum() == 0:
            continue

        user_data['dcg'] = user_data['is_relevant'] / np.log2(user_data.index + 2)
        dcg = user_data['dcg'].sum()

        idcg = sum(1 / np.log2(i + 2) for i in range(int(user_data['is_relevant'].sum())))
        ndcg_scores.append(dcg / idcg if idcg > 0 else 0)

    return sum(ndcg_scores) / len(ndcg_scores) if ndcg_scores else 0

# Calculate Metrics
precision, recall = precision_recall_at_k(predictions, k=5)
map_score = mean_average_precision(predictions, k=5)
ndcg_score = ndcg_at_k(predictions, k=5)

# Display Metrics
print(f"Precision@5: {precision}")
print(f"Recall@5: {recall}")
print(f"MAP@5: {map_score}")
print(f"NDCG@5: {ndcg_score}")


Precision@5: 0.75
Recall@5: 1.0
MAP@5: 1.0
NDCG@5: 0.8154648767857288


<ipython-input-98-8812b2b7b36b>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_data['dcg'] = user_data['is_relevant'] / np.log2(user_data.index + 2)
<ipython-input-98-8812b2b7b36b>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_data['dcg'] = user_data['is_relevant'] / np.log2(user_data.index + 2)


In [51]:
import pickle

In [52]:
with open("model_knn.pkl","wb") as f:
    pickle.dump(model, f)

In [53]:
with open("model_knn.pkl", "rb") as f:
    loaded_model = pickle.load(f)


In [54]:
!pip install streamlit

In [55]:
!pip install pyngrok

In [56]:
!wget -q -O - ipv4.icanhazip.com


35.237.116.74


In [58]:
%%writefile app.py
import streamlit as st
st.write("hi sandy")

Overwriting app.py


In [99]:
! streamlit run app.py & npx localtunnel --port 8501




⠙⠹
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.237.116.74:8501

⠸⠼⠴⠦⠧⠇your url is: https://free-suits-sit.loca.lt
  Stopping...
^C


In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok...


--2025-01-22 06:37:46--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok...
Resolving bin.equinox.io (bin.equinox.io)... 99.83.220.108, 13.248.244.96, 75.2.60.68, ...
Connecting to bin.equinox.io (bin.equinox.io)|99.83.220.108|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-01-22 06:37:46 ERROR 404: Not Found.



In [ ]:
import streamlit as st
import pandas as pd
from surprise import Dataset, Reader, KNNBasic
import pickle

# Load pre-trained KNN model
with open("gs.pkl", "rb") as f:
    model = pickle.load(f)


# Load dataset
df = pd.read_csv('Loan_data_k.csv')

# Mapping product IDs to product names
product_mapping = {
    "P001": "Savings Account", "P002": "Premium Checking Account", "P003": "Credit Card", 
    "P004": "Personal Loan", "P005": "Home Loan", "P006": "Auto Loan", "P007": "Business Loan", 
    "P008": "Student Loan", "P009": "Investment Fund", "P010": "Retirement Plan", "P011": "Insurance Policy", 
    "P012": "Mutual Fund", "P013": "Bond", "P014": "Certificate of Deposit", "P015": "Home Equity Line", 
    "P016": "Mortgage Refinance", "P017": "Business Credit Line", "P018": "Auto Refinance", 
    "P019": "Home Improvement Loan", "P020": "Gold Loan", "P021": "Cash Credit", "P022": "Short-Term Loan", 
    "P023": "Long-Term Loan", "P024": "Travel Loan", "P025": "Medical Loan", "P026": "Emergency Loan", 
    "P027": "Holiday Loan", "P028": "Debt Consolidation Loan", "P029": "Small Business Loan", 
    "P030": "Agricultural Loan", "P031": "Technology Loan", "P032": "Education Savings Plan", 
    "P033": "Wealth Management", "P034": "Stock Investment", "P035": "Real Estate Investment", 
    "P036": "International Investment", "P037": "Fixed Deposit", "P038": "Recurring Deposit", 
    "P039": "Loan Against Property", "P040": "Gold Investment", "P041": "Retirement Savings", 
    "P042": "High-Yield Savings Account", "P043": "Money Market Account", "P044": "Insurance Savings", 
    "P045": "Pension Plan", "P046": "Child Education Fund", "P047": "Healthcare Savings", 
    "P048": "Property Investment", "P049": "Auto Insurance"
}

df["Product_name"] = df["Product_id"].map(product_mapping)

# Mapping interaction types
interaction_type_mapping = {'Viewed': 1, 'Clicked': 2, 'Purchased': 3}
df["Interaction_Type"] = df["Interaction_type"].map(interaction_type_mapping)

# Surprise Reader for Surprise model compatibility
reader = Reader(rating_scale=(1, 3))

# Recommendation function
def recommend_products(customer_id, model, interaction_data, product_mapping, n=5):
    all_products = set(interaction_data['Product_id'].unique())

    # Get customer's interacted products
    interacted_products = set(interaction_data[interaction_data['Customer_id'] == customer_id]['Product_id'])
    print(f"Customer {customer_id} has interacted with: {interacted_products}")  # Debugging line

    # If the customer has no interactions, suggest popular products from all available products
    products_to_predict = list(all_products - interacted_products) if customer_id in interaction_data['Customer_id'].values else list(all_products)

    print(f"Products to predict for customer {customer_id}: {products_to_predict}")  # Debugging line

    # If no products to predict, return empty
    if not products_to_predict:
        return pd.DataFrame(columns=['Product_Id', 'Product_Name'])

    # Predictions for each product
    predictions = [model.predict(customer_id, product_id) for product_id in products_to_predict]
    
    print(f"Predictions for customer {customer_id}: {predictions}")  # Debugging line
    
    # Sort predictions by estimated rating (highest first)
    top_n_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)[:n]
    
    # Get top recommended product IDs and names
    recommended_product_ids = [pred.iid for pred in top_n_predictions]
    recommended_products = pd.DataFrame({
        'Product_Id': recommended_product_ids,
        'Product_Name': [product_mapping.get(pid, 'Unknown') for pid in recommended_product_ids]
    })
    return recommended_products

# Sidebar Navigation
options = ["PRODUCT RECOMMENDATIONS"]
selected = st.sidebar.selectbox("Select a page", options)

# Check if "PRODUCT RECOMMENDATIONS" page is selected
if selected == "PRODUCT RECOMMENDATIONS":
    col1, col2, col3 = st.columns([1, 2, 1])

    with col2:
        st.subheader("PRODUCT RECOMMENDATIONS")

        # Input field for Customer ID
        customer_id = st.text_input("Enter Customer ID", "")

        # Button to get recommendations
        if st.button("Get Recommendations"):
            if customer_id:
                if customer_id in df['Customer_id'].values:
                    recommendations = recommend_products(customer_id, model, df, product_mapping)

                    if not recommendations.empty:
                        st.markdown(
                            f"""
                            <div style="background-color: #d9f9d9; padding: 10px; border-radius: 10px;">
                                <h3 style="color: #000000; text-align: center;">
                                    Top Recommended Products for {customer_id}:
                                </h3>
                            </div>
                            """,
                            unsafe_allow_html=True
                        )
                        st.dataframe(recommendations)
                    else:
                        st.write("No recommendations available.")
                else:
                    st.error("Invalid Customer ID. Please enter a valid Customer ID.")
            else:
                st.warning("Please enter a valid Customer ID.")
